In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
def sigmoid(x, derivative=False):
    return 1 / (1 + np.exp(-x)) if not derivative else x * (1 - x)

def tanh(x, derivative=False):
    return np.tanh(x) if not derivative else 1 - x * x

def relu(x, derivative=False):
    if not derivative:
        output = np.copy(x)
        output[output < 0] = 0
        return output

    if derivative:
        output = np.copy(x)
        output[output > 0] = 1
        output[output <= 0] = 0
        return output

func_dict = {}
func_dict['sigmoid'] = sigmoid
func_dict['tanh'] = tanh
func_dict['relu'] = relu

In [ ]:
class CPCLSNet(object):

    def __init__(self, input_size, output_size, hidden_size, sample_num):
        self.n = input_size
        self.q = output_size
        self.p = hidden_size
        self.m = sample_num
        self.delta_p = 1
        self.X = []
        self.Y = []
        self.H = []
        self.W = []
        self.B = []

        self.f = func_dict['sigmoid']
        self.hlayer_num = 0

    def set_data(self, x_train, y_train, x_test, y_test):
        self.X.append(x_train.tolist())
        self.Y.append(y_train.tolist())
        self.x_test = x_test
        self.y_test = y_test

    def cal_ih_weight(self):
        print("p:{}".format(self.p))
        x = np.array(self.X[self.hlayer_num])
        S = np.dot((x - np.mean(x, axis=0)).T, (x - np.mean(x, axis=0)))
        S = S / (self.m - 1)
        print("S shape:{}".format(S.shape))
        # 增加复数判断，直接取实数部分。
        eigenval = np.linalg.eig(S)[0]
        if eigenval.dtype=='complex128':
            eigenval = eigenval.real
        big0 = np.where(eigenval>1e-2,1,0)
        
        biggest = np.sum(big0)
        print(biggest)
        if biggest<self.delta_p:
            self.delta_p = biggest
        eigenvec = np.linalg.eig(S)[1]

        # select highest p eigenvalue & corresponding eigenvector
        eigenval_sort = np.argsort(eigenval)
#         print(eigenval)
#         print(eigenval[eigenval_sort[-self.p:]])
        W = eigenvec[eigenval_sort[-self.p:]].T
        if W.dtype=='complex128':
            W = W.real

        self.W.append(W.tolist())
        

    def cal_hidden_out(self):
        x = np.array(self.X[self.hlayer_num])
        w = np.array(self.W[self.hlayer_num])
#         out = np.dot(x, w)
#         print("out shape:{}".format(out.shape))
        H = np.dot(x, w)
#         H = self.f(out)

        return H

    def update_HX(self):
        newH = self.cal_hidden_out().tolist()
        oldx = np.array(self.X[self.hlayer_num])
        newx = np.hstack((oldx,newH))
        self.X.append(newx.tolist())
        self.H.append(newH)

    def cal_ho_weight(self):
        h = np.array(self.H[-1])
        y = np.array(self.Y[0])
        beta1 = np.linalg.inv(np.dot(h.T, h))
        beta2 = np.dot(beta1, h.T)
        beta = np.dot(beta2, y)
        self.B.append(beta.tolist())

    def cal_out_mat(self):
        h = np.array(self.H[-1])
        b = np.array(self.B[-1])
        y_hat = np.dot(h, b)

        return y_hat

    def update_p(self):
        self.p += self.delta_p
        
    def forward(self):
        x = np.array(self.x_test)
        b = np.array(self.B[-1])
        for i in range(self.hlayer_num):
            w = np.array(self.W[i])
#             out = np.dot(x, w)   
            h = np.dot(x, w)
#             h = self.f(out)
            x = np.hstack((x,h.tolist()))
        y_pred = np.dot(h,b)
        
        return y_pred
    
    
    def train(self):
        loss=[]
        loss_test=[]
        e=999
        for i in range(50):
            if e < 6e-5:
                break
            print("\n#####{}#####".format(i))
            self.cal_ih_weight()
            self.update_HX()
            self.cal_ho_weight()
            y_hat = self.cal_out_mat()
            e = np.mean((y_hat - self.Y) ** 2)
            print("error:{:.8f}".format(e))
            self.update_p()
            self.hlayer_num+=1
            i+=1
            y_pred = self.forward()
            l = np.mean((y_pred - self.y_test) ** 2)
            print(l)
            loss.append(e)
            loss_test.append(l)
        return loss,loss_test
            
        

In [8]:
from sklearn.preprocessing import MinMaxScaler

train_data = pd.read_csv('../Dataset/Denoicing/Train.csv')
test_data = pd.read_csv('../Dataset/Denoicing/Test.csv')
_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

sc_x_train = MinMaxScaler(feature_range=(0, 1))
sc_y_train = MinMaxScaler(feature_range=(0, 1))
_data_x = _data.drop(labels=['FF', 'FF1C', 'FF2C'], axis=1).copy()
sc_x_train.fit(_data_x)
_data_y = _data[['FF']].copy()
sc_y_train.fit(_data_y)

X_train = train_data.drop(labels=['FF', 'FF1C', 'FF2C'], axis=1).copy()
x_train = sc_x_train.transform(X_train)

Y_train = train_data[['FF']].copy()
y_train = sc_y_train.transform(Y_train)

X_test = test_data.drop(labels=['FF', 'FF1C', 'FF2C'], axis=1).copy()
x_test = sc_x_train.transform(X_test)

Y_test = test_data[['FF']].copy()
y_test = sc_y_train.transform(Y_test)

In [9]:
input_size, output_size, sample_size = x_train.shape[1], y_train.shape[1], x_train.shape[0]
print(input_size,output_size,sample_size)

16 1 1961


In [ ]:
cpclsnet = CPCLSNet(input_size,output_size,16,sample_size)
cpclsnet.set_data(x_train, y_train, x_test, y_test)
loss, loss_test = cpclsnet.train()